# Generate the data for Neural Network

# Packages and general functions

In [ ]:
#include("D:/Dox/Recherche/Julia/OQLiege.jl");
#include("D:/Dox/Recherche/Julia/ACLiege.jl");
include("/home/jdenis_oq/Documents/Recherche/Julia/OQLiege.jl");
include("/home/jdenis_oq/Documents/Recherche/Julia/ACLiege.jl");

#import Pkg
using Distributed
using Statistics
using Combinatorics

cd(@__DIR__)

# Compute the Husimi function of a spin state in the Dicke basis
function dktoHusimi(ψ::Vector{ComplexF64}, nSteps::Int64)
    N = length(ψ)-1
    Θ = LinRange(0.0,π,nSteps)
    Φ = LinRange(0.0,2π,nSteps)
    husimi = Matrix{Float64}(undef,nSteps,nSteps)
    for i in 1:nSteps
        for j in 1:nSteps
            husimi[i,j] = abs(dot(Coherentdk(N,Θ[i],Φ[j]),ψ))^2
        end
    end
    return husimi
end

#Compute the maximum value of the Husimi function of a spin state in the Dicke basis
function Qmax(ψ::Vector{ComplexF64}, nSteps::Int64)
    husimi = dktoHusimi(ψ,nSteps)
    return maximum(vec(husimi))
end

function θϕtoMajroots(θϕ::Matrix{Float64})
    N = size(θϕ,1)
    roots = Vector{ComplexF64}(undef,N)
    [roots[i] = cot(θϕ[i,1]/2)*exp(-θϕ[i,2]*im) for i in 1:N]
    return roots
end

function θϕtodk(θϕ::Matrix{Float64})
    N = size(θϕ,1)
    roots = θϕtoMajroots(θϕ)
    dk = Vector{ComplexF64}(undef,N+1)
    [dk[k+1] = (-1)^k*(fromroots(roots)[k])/sqrt(binomial(N,k)) for k in 0:N]
    return dk/norm(dk)
end

# Random Wehrl moments

In [ ]:
# Generate the data based on the GEdk function with all the partitions of collapsed Majorana points possible
function generate_data(N::Int64, pmax::Int64)

	converged = false
	local GE,ψ

	while !converged
        parts = collect(partitions(N))
		part = parts[rand(1:length(parts))]

		if length(part) == 1
			ψ = Coherentdk(N,pi*rand(),2*pi*rand())
		else
			ψ = randstate(N+1)
			angles = dktoθϕ(ψ)
			j = 0
			for i in 1:length(part)
				nCollapsed = part[i]
				for k in 2:nCollapsed
					angles[j+k,:] = angles[1+j,:]
				end
				j += nCollapsed
			end
			ψ = θϕtodk(angles)
		end
		GE, converged = GEdk(ψ,10)
	end
	moments = Vector{Float64}(undef,pmax)
	Threads.@threads for i in 1:pmax
		moments[i] = Wehrl_moment_dk(ψ, i)
	end

	return ψ,moments,1-GE
end

function generate_data(N::Int64, pmax::Int64)

	converged = false
	local GE,ψ

	@time while !converged
        α = rand()
		k = rand(1:N)
    	ψ = normalize(α * GHZdk(N) + (1-α) * Dickedk(k,N))
		GE, converged = GEdk(ψ,10)
	end
	moments = Vector{Float64}(undef,pmax)
	@time Threads.@threads for i in 1:pmax
		moments[i] = Wehrl_moment_dk(ψ, i)
	end

	return ψ,moments,1-GE
end

# Store the Wehrl moments and maximum of the Husimi function from random states
for N in 2:10
	println(N)
	pmax = 15
	for i in 1:10000
		println(i)
		ψ, moments, qmax = generate_data(N,pmax)
		open("Data/WehrlMomentTrainStates_"*string(N)*".dat", "a") do io
			writedlm(io, [transpose(real(ψ)) transpose(imag(ψ))])
		end
		open("Data/WehrlMomentTrainData_"*string(N)*".dat", "a") do io
			writedlm(io, [transpose(moments) qmax])
		end
		ψ, moments, qmax = generate_data(N,pmax)
		open("Data/WehrlMomentTestStates_"*string(N)*".dat", "a") do io
			writedlm(io, [transpose(real(ψ)) transpose(imag(ψ))])
		end
		open("Data/WehrlMomentTestData_"*string(N)*".dat", "a") do io
			writedlm(io, [transpose(moments) qmax])
		end
	end
end

In [ ]:
N=3
α = rand()
k = rand(1:N)
ψ = normalize(α * GHZdk(N) + (1-α) * Dickedk(k,N))
GE, converged = GEdk(ψ,10)
GE

# Random States Distance moments

In [ ]:
# Distance between two points on a sphere of radius r measured along the surface of the sphere.
# The points have longitudes ϕ1 and ϕ2 and latitudes θ1 and θ2
# https://en.wikipedia.org/wiki/Great-circle_distance
function SphericalDistance(r::Float64, θ1::Float64,θ2::Float64, ϕ1::Float64, ϕ2::Float64)
    return r*acos(sin(θ1)sin(θ2)+cos(θ1)cos(θ2)cos(ϕ1-ϕ2))
end

# Compute the moments from the distance between the points on the Majorana sphere
function DistanceMoments(distances::Vector{Float64}, pmax::Int64)
    moments = Vector{Float64}(undef,pmax)
    for i in 1:pmax
        if i == 1
            moments[i] = mean(distances)
        else
            moments[i] = sum((distances .- mean(distances)).^i)/(length(distances)-1)
        end
    end
    return moments
end

function generate_data(N::Int, pmax::Int64)

    converged = false
	local qmax,ψ

	while !converged
		ψ = randstate(N+1)
		GE, converged = GEdk(ψ,10)
		qmax = 1-GE
	end
    angles = dktoθϕ(ψ)
    distances = Vector{Float64}(undef,round(Int,N*(N-1)/2))
    index = 0
    for i in 1:N
        for j in i+1:N
            index += 1
            distances[index] = SphericalDistance(1.0, angles[i,1], angles[j,1], angles[i,2], angles[j,2])
        end
    end
    moments = DistanceMoments(distances,pmax)
    return ψ, moments, qmax
end

# Store the distance moments and maximum of the Husimi function from random states
for N in 2:20
    pmax = 20
    for i in 1:10000
        println(i)
        @time ψ,moments,qmax = generate_data(N,pmax)
        open("DistanceMomentsTrainStates_"*string(N)*".dat", "a") do io
			writedlm(io, [transpose(real(ψ)) transpose(imag(ψ))])
		end
        open("DistanceMomentsTrainData_"*string(N)*".dat", "a") do io
            writedlm(io, [transpose(moments) qmax])
        end

        @time ψ,moments,qmax = generate_data(N,pmax)
        open("DistanceMomentsTestStates_"*string(N)*".dat", "a") do io
			writedlm(io, [transpose(real(ψ)) transpose(imag(ψ))])
		end
        open("DistanceMomentsTestData_"*string(N)*".dat", "a") do io
            writedlm(io, [transpose(moments) qmax])
        end
    end
end

# Husimi function

In [ ]:
function generate_data(N::Int)

    converged = false
	local qmax,ψ

	while !converged
		ψ = randstate(N+1)
		GE, converged = GEdk(ψ,10)
		qmax = 1-GE
	end

    husimi = dktoHusimi(ψ,round(Int,log(N^3)))
    return ψ, vec(husimi), qmax
end

for N in 2:15
    for i in 1:10000
        println(i)

        @time ψ,husimi,qmax = generate_data(N)
        open("HusimiTrainStates_"*string(N)*".dat", "a") do io
			writedlm(io, [transpose(real(ψ)) transpose(imag(ψ))])
		end
        open("HusimiTrainData_"*string(N)*".dat", "a") do io
            writedlm(io, [transpose(husimi) qmax])
        end

        @time ψ, husimi,qmax = generate_data(N)
        open("HusimiTestStates_"*string(N)*".dat", "a") do io
			writedlm(io, [transpose(real(ψ)) transpose(imag(ψ))])
		end
        open("HusimiTestData_"*string(N)*".dat", "a") do io
            writedlm(io, [transpose(husimi) qmax])
        end
    end
end

# Degenerate Majorana points states

In [ ]:
N = 10
nDegeneratePoints = 1
psi = randstate(N+1);
points = MajPoints(psi);
[points[i][end-j] = points[i][end-j+1] for i in 1:3 for j in 1:nDegeneratePoints-1];
points

Vérifions que la fonctionne 1-GEdk donne bien la fonction de Husimi.

In [ ]:
using GLMakie
include("/home/jdenis_oq/Documents/Recherche/Julia/OQLiege.jl");

function dktoHusimi(ψ::Vector{ComplexF64}, nSteps::Int64)
    N = length(ψ)-1
    Θ = LinRange(0.0,π,nSteps)
    Φ = LinRange(0.0,2π,nSteps)
    husimi = Matrix{Float64}(undef,nSteps,nSteps)
    for i in 1:nSteps
        for j in 1:nSteps
            husimi[i,j] = abs(dot(Coherentdk(N,Θ[i],Φ[j]),ψ))^2
        end
    end
    return husimi
end

function Qmax(ψ::Vector{ComplexF64}, nSteps::Int64)
    husimi = dktoHusimi(ψ,nSteps)
    return maximum(vec(husimi))
end
@time GEdk(randstate(11),10)
@time Qmax(randstate(11),64)
psis = [randstate(11) for i in 1:100]
test1 = 1 .- GEdk.(psis,10)
test2 = Qmax.(psis,64)
fig = scatter(test1,test2, markersize = 5);
lines!(0.0:0.1:1.0,0.0:0.1:1.0);
fig